## Load data.

In [1]:
import numpy as np
import os
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
from itertools import combinations
from scipy import stats

from helper_functions import stats, pval_star, SENT2LAB, SENT2IDX, SENT_COLORS, assert_in_xylims, set_xticks, set_yticks, get_xylims_noerr, print_sigfig

CWD = os.path.abspath("")  # Jupyter notebook path.
dir_input = os.path.join(CWD, "input")  # For params.
dir_dict = os.path.join(CWD, "dicts")  # Data to plot.
dir_output = os.path.join(CWD, "output")  # Folder to put figures in.
dir_npy = os.path.join(CWD, "npy")  # Data files needed for plotting figures.

# Set up plotting parameters.
font_kw = {"family": "arial", "weight": "normal", "size": "7"}  # Markers and such.
mpl.rc("font", **font_kw)

# Load data to plot.
USE_CAPS = True  # Capitalize genders.
BY = "Gender"  # Used in figure file names.

ratio_mat_rel = np.load(os.path.join(dir_npy, "ratio_mat_rel-gender_effect.npy"))
genders_select = np.load(os.path.join(dir_npy, "groups-gender_effect.npy"))

with open(os.path.join(dir_input, "params.json")) as f:
    params = json.load(f)

## Make figures.

In [2]:
def plot_gender(rmrs, grps_subset, dir_, by=None):

    fig, axes = plt.subplots(2, 2, figsize=(3.41 * 2, 3.41 * 2))
    for ii, sent in enumerate([-1, 1]):
        color_ = SENT_COLORS[SENT2IDX[sent]]
        assert grps_subset[0] == "man" and grps_subset[1] == "woman", f"{grps_subset} is not 'man, woman'"
        # Mean is towards all collab distance (except self, which is distance 0).
        x_mean_M_a = rmrs[0, 0, -1, SENT2IDX[sent], :]
        x_mean_W_a = rmrs[0, 1, -1, SENT2IDX[sent], :]
        x_mean_M_M = rmrs[0, 0, 0, SENT2IDX[sent], :]
        x_mean_W_W = rmrs[0, 1, 1, SENT2IDX[sent], :]
        x_mean_M_W = rmrs[0, 0, 1, SENT2IDX[sent], :]
        x_mean_W_M = rmrs[0, 1, 0, SENT2IDX[sent], :]

        # Bias is non-collab - collab, regardless of sentiment type.
        x_bias_M_a = rmrs[2, 0, -1, SENT2IDX[sent], :] - rmrs[1, 0, -1, SENT2IDX[sent], :]
        x_bias_W_a = rmrs[2, 1, -1, SENT2IDX[sent], :] - rmrs[1, 1, -1, SENT2IDX[sent], :]
        x_bias_M_M = rmrs[2, 0, 0, SENT2IDX[sent], :] - rmrs[1, 0, 0, SENT2IDX[sent], :]
        x_bias_W_W = rmrs[2, 1, 1, SENT2IDX[sent], :] - rmrs[1, 1, 1, SENT2IDX[sent], :]
        x_bias_M_W = rmrs[2, 0, 1, SENT2IDX[sent], :] - rmrs[1, 0, 1, SENT2IDX[sent], :]
        x_bias_W_M = rmrs[2, 1, 0, SENT2IDX[sent], :] - rmrs[1, 1, 0, SENT2IDX[sent], :]

        xticklabels = ["M", "W", "M-M", "W-W", "M-W", "W-M"]
        kwargs = dict(
            widths=0.5,
            boxprops={"color": color_, "linewidth": 0.5},
            medianprops={"color": color_, "linewidth": 0.5},
            whiskerprops={"color": color_, "linewidth": 0.5, "visible": True},
            capprops={"color": color_, "linewidth": 0.5},
        )
        kwargs.update(dict(showmeans=False, showfliers=False, showbox=True, showcaps=True))

        ax = axes[0, ii]  # Mean row.
        ax.boxplot([x_mean_M_a, x_mean_W_a, x_mean_M_M, x_mean_W_W, x_mean_M_W, x_mean_W_M], **kwargs)

        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        ax.set_xticks(range(1, len(xticklabels) + 1), xticklabels)
        ax.plot(xlim, [0, 0], color="grey", alpha=0.5, zorder=1, linestyle=":")  # Baseline (indistinguishable from null).
        ax.axvline(x=2.5, color="black", alpha=1, zorder=99, linestyle="-", linewidth=0.8)
        ax.axvline(x=4.5, color="black", alpha=1, zorder=99, linestyle="-", linewidth=0.8)
        ax.set_ylabel(f"{SENT2LAB[sent]} Mean")
        ax.spines[["right", "top"]].set_visible(False)
        ax.set_xlim(xlim)
        if sent == -1:
            ax.set_ylim(params["ylim_NEG"])
        if sent == 1:
            ax.set_ylim(params["ylim_POS"])

        ax = axes[1, ii]  # Bias row.
        xticklabels = ["M", "W", "M-M", "W-W", "M-W", "W-M"]
        ax.boxplot([x_bias_M_a, x_bias_W_a, x_bias_M_M, x_bias_W_W, x_bias_M_W, x_bias_W_M], **kwargs)

        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        ax.set_xticks(range(1, len(xticklabels) + 1), xticklabels)
        ax.plot(xlim, [0, 0], color="grey", alpha=0.5, zorder=1, linestyle=":")  # Baseline (indistinguishable from null).
        ax.axvline(x=2.5, color="black", alpha=1, zorder=99, linestyle="-", linewidth=0.8)
        ax.axvline(x=4.5, color="black", alpha=1, zorder=99, linestyle="-", linewidth=0.8)
        ax.set_ylabel(f"{SENT2LAB[sent]} Bias")
        ax.spines[["right", "top"]].set_visible(False)
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)

    for ax in axes.flat:
        set_yticks(ax)
        major_len = 6.5
        major_width = 1.5
        ax.tick_params(axis="x", which="major", length=major_len, width=major_width, labelsize=10)

    fig.tight_layout()
    plt.show()
    fig.savefig(os.path.join(dir_, f"{by} Mean-Bias.svg"), bbox_inches="tight", transparent=True)
    fig.clf()  # Clear figure.
    plt.close(fig=fig)  # Close figure.

In [ ]:
plot_gender(ratio_mat_rel, genders_select, dir_output, by=BY)

### t-tests

In [ ]:
gen = [x[0].upper() for x in genders_select]
gen2idx = {g: i for i, g in enumerate(gen)}
idx2gen = {i: g for i, g in enumerate(gen)}
rmrs = ratio_mat_rel


def nonce(arr):
    return f"{np.mean(arr):>5.2f}±{np.std(arr, ddof=1):.2f}"


for sent in [-1, 1]:
    print(f"\n{SENT2LAB[sent]} Mean")

    arri = rmrs[0, 0, -1, SENT2IDX[sent], :]
    arrj = rmrs[0, 1, -1, SENT2IDX[sent], :]
    abc1 = np.mean(arri)
    abc2 = np.std(arri, ddof=1)
    abc3 = np.mean(arrj)
    abc4 = np.std(arrj, ddof=1)
    res = stats.ttest_ind(arri, arrj, equal_var=False, alternative="two-sided")
    # txt_box0 = f"{idx2gen[0]} {print_sigfig(abc1)}±{print_sigfig(abc2)} {idx2gen[1]} {print_sigfig(abc3)}±{print_sigfig(abc4)}"
    versus = f"{idx2gen[0]} v {idx2gen[1]}"
    ci95 = f" 95%CI ({print_sigfig(res.confidence_interval().low)},{print_sigfig(res.confidence_interval().high)})"
    txt_box0 = f"{versus:<9} {print_sigfig((abc1-abc3)):>6} {ci95:<20}"
    txt_box0 += f" t={print_sigfig(res.statistic)} {pval_star(res.pvalue)} df={print_sigfig(res.df)}"
    print(txt_box0)

    pair_dict = {(ii, jj): rmrs[0, ii, jj, SENT2IDX[sent], :] for ii in gen2idx.values() for jj in gen2idx.values()}
    for pi, pj in combinations(pair_dict.keys(), 2):
        arri = pair_dict[pi]
        arrj = pair_dict[pj]
        abc1 = np.mean(arri)
        abc2 = np.std(arri, ddof=1)
        abc3 = np.mean(arrj)
        abc4 = np.std(arrj, ddof=1)
        res = stats.ttest_ind(arri, arrj, equal_var=False, alternative="two-sided")
        # txt_box0 = f"{idx2gen[pi[0]]}-{idx2gen[pi[1]]} {print_sigfig(abc1)}±{print_sigfig(abc2)} {idx2gen[pj[0]]}-{idx2gen[pj[1]]} {print_sigfig(abc3)}±{print_sigfig(abc4)}"
        versus = f"{idx2gen[pi[0]]}-{idx2gen[pi[1]]} v {idx2gen[pj[0]]}-{idx2gen[pj[1]]}"
        ci95 = f" 95%CI ({print_sigfig(res.confidence_interval().low)},{print_sigfig(res.confidence_interval().high)})"
        txt_box0 = f"{versus:<9} {print_sigfig((abc1-abc3)):>6} {ci95:<20}"
        txt_box0 += f" t={print_sigfig(res.statistic)} {pval_star(res.pvalue)} df={print_sigfig(res.df)}"
        print(txt_box0)


for sent in [-1, 1]:
    print(f"\n{SENT2LAB[sent]} Bias")

    arri = rmrs[2, 0, -1, SENT2IDX[sent], :] - rmrs[1, 0, -1, SENT2IDX[sent], :]
    arrj = rmrs[2, 1, -1, SENT2IDX[sent], :] - rmrs[1, 1, -1, SENT2IDX[sent], :]
    abc1 = np.mean(arri)
    abc2 = np.std(arri, ddof=1)
    abc3 = np.mean(arrj)
    abc4 = np.std(arrj, ddof=1)
    res = stats.ttest_ind(arri, arrj, equal_var=False, alternative="two-sided")
    # txt_box0 = f"{idx2gen[0]} {print_sigfig(abc1)}±{print_sigfig(abc2)} {idx2gen[1]} {print_sigfig(abc3)}±{print_sigfig(abc4)}"
    versus = f"{idx2gen[0]} v {idx2gen[1]}"
    ci95 = f" 95%CI ({print_sigfig(res.confidence_interval().low)},{print_sigfig(res.confidence_interval().high)})"
    txt_box0 = f"{versus:<9} {print_sigfig((abc1-abc3)):>6} {ci95:<20}"
    txt_box0 += f" t={print_sigfig(res.statistic)} {pval_star(res.pvalue)} df={print_sigfig(res.df)}"
    print(txt_box0)

    pair_dict = {
        (ii, jj): rmrs[2, ii, jj, SENT2IDX[sent], :] - rmrs[1, ii, jj, SENT2IDX[sent], :]
        for ii in gen2idx.values()
        for jj in gen2idx.values()
    }
    for pi, pj in combinations(pair_dict.keys(), 2):
        arri = pair_dict[pi]
        arrj = pair_dict[pj]
        abc1 = np.mean(arri)
        abc2 = np.std(arri, ddof=1)
        abc3 = np.mean(arrj)
        abc4 = np.std(arrj, ddof=1)
        res = stats.ttest_ind(arri, arrj, equal_var=False, alternative="two-sided")
        # txt_box0 = f"{idx2gen[pi[0]]}-{idx2gen[pi[1]]} {print_sigfig(abc1)}±{print_sigfig(abc2)} {idx2gen[pj[0]]}-{idx2gen[pj[1]]} {print_sigfig(abc3)}±{print_sigfig(abc4)}"
        versus = f"{idx2gen[pi[0]]}-{idx2gen[pi[1]]} v {idx2gen[pj[0]]}-{idx2gen[pj[1]]}"
        ci95 = f" 95%CI ({print_sigfig(res.confidence_interval().low)},{print_sigfig(res.confidence_interval().high)})"
        txt_box0 = f"{versus:<9} {print_sigfig((abc1-abc3)):>6} {ci95:<20}"
        txt_box0 += f" t={print_sigfig(res.statistic)} {pval_star(res.pvalue)} df={print_sigfig(res.df)}"
        print(txt_box0)